# Texas Public School Discipline Datasets 2017-2018

This code creates ranking datasets for all Texas public schools and districts by discipline types. 
* Rankings are based on the percentage of discipline actions compared to campus cumulative year end enrollment.  
* A school with the lowest rank percentage has the highest percentage of a particular discipline type in the State of Texas or in a school district. 
* Data used to create the rankings datasets comes from: https://rptsvr1.tea.texas.gov/adhocrpt/Disciplinary_Data_Products/Download_All_Campuses.html
* There were 8,612 campuses considered in this study.
* * There were 1200 school districts considered in this study.

Prepared by:  Jake Drew Ph.D.

In [67]:
#import required Libraries
import pandas as pd
import numpy as np
import os
import string

In [68]:
#Read in raw data file for disipline
path = 'D:/Nathan/HebronSchoolData/RawData/CAMPUS_summary_18_NO_HEADERS.csv'
discipline2018 = pd.read_csv(path, low_memory=False) #, dtype={'unit_code': object}) 

#Remove all masking
discipline2018['YR18'] = discipline2018['YR18'].str.lstrip('<')

#Remove or delete -999 records
#discipline2018 = discipline2018.replace({"-999":0})
discipline2018 = discipline2018[discipline2018['YR18'] != "-999"]

#Convert counts to int
discipline2018['YR18'] = discipline2018['YR18'].astype(int)

## Data Preparation
* Data used in this report comes from: https://rptsvr1.tea.texas.gov/adhocrpt/Disciplinary_Data_Products/Download_All_Campuses.html
* TEA report headers are deleted before reading in the data. 
* Records with -999 values are deleted.
* We remove the '<' character from masked records using the remaining number as an estimated / imputed value for these datasets.

### Steps to Create a Dataset for Discipline Rankings
* First, the All Campus Discipline Report is filtered to review only the A-PARTICIPATION and B-DISCIPLINE DATA TRENDS section headings. 
* Next the filtered table is pivoted to create new columns for each unique heading in the filtered sections.
* Finally, the school metadata (Region, District, and Campus Values) are added / merged back to the pivoted data to create a new ranking dataset with one record per public school campus and all unique A-PARTICIPATION and B-DISCIPLINE DATA TRENDS section heading counts as columns in the final dataset.   

In [69]:
#Get a unique list of each school's metadata
schools2018 = discipline2018[['REGION', 'DISTRICT NAME AND NUMBER', 'CAMPUS NAME AND NUMBER']]
schools2018 = schools2018.drop_duplicates()
schools2018.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8614 entries, 0 to 328583
Data columns (total 3 columns):
REGION                      8614 non-null int64
DISTRICT NAME AND NUMBER    8614 non-null object
CAMPUS NAME AND NUMBER      8614 non-null object
dtypes: int64(1), object(2)
memory usage: 269.2+ KB


In [70]:
#Filter displine data to review only the A-PARTICIPATION and B-DISCIPLINE DATA TRENDS sections
disciplineStats2018 = discipline2018[ (discipline2018['SECTION'] == 'A-PARTICIPATION') |
                                      (discipline2018['SECTION'] == 'B-DISCIPLINE DATA TRENDS')
                                    ]

disciplineStats2018 = disciplineStats2018[['CAMPUS NAME AND NUMBER',
                                           'HEADING NAME', 
                                           'YR18']]

In [71]:
#Pivot table creating one record per school campus with cols for all section heading names / discipline categories
disciplineStats2018 = pd.pivot_table(disciplineStats2018
                                     , values=  'YR18'
                                     , index=   'CAMPUS NAME AND NUMBER'
                                     , columns= 'HEADING NAME')

#Make our index a column for merges later
disciplineStats2018.reset_index(inplace=True) 

In [72]:
#Look at the new discipline stats columns we created during the table pivot
disciplineStats2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8612 entries, 0 to 8611
Data columns (total 15 columns):
CAMPUS NAME AND NUMBER                   8612 non-null object
CAMPUS CUMULATIVE YEAR END ENROLLMENT    8490 non-null float64
CAMPUS DISCIPLINE POPULATION             5777 non-null float64
CAMPUS DISCIPLINE RECORD COUNT           6341 non-null float64
COUNT OF STUDENTS EXPELLED               62 non-null float64
COUNT OF STUDENTS EXPELLED TO JJAEP      44 non-null float64
COUNT OF STUDENTS REMOVED TO A DAEP      1730 non-null float64
COUNT OF STUDENTS SUSPENDED IN SCHOOL    4945 non-null float64
DISCRETIONARY DAEP REMOVALS              842 non-null float64
DISCRETIONARY EXPULSIONS                 21 non-null float64
DISCRETIONARY EXPULSIONS TO JJAEP        13 non-null float64
MANDATORY DAEP REMOVALS                  778 non-null float64
MANDATORY EXPULSIONS                     6 non-null float64
MANDATORY EXPULSIONS TO JJAEP            4 non-null float64
STUDENTS SUSPENDED OUT OF SC

In [73]:
#Merge school metadata back to discipline category data  
print('*********************************Start: disciplineStats2018 Data*********************************')
disciplineStats2018.info(verbose=False)

#Merge profile and profileMetric data
disciplineStats2018 = disciplineStats2018.merge(schools2018,how='left',on='CAMPUS NAME AND NUMBER', suffixes=('', '_Drop'))

print('*********************************After: disciplineStats2018 Data**************************')
disciplineStats2018.info(verbose=False)

*********************************Start: disciplineStats2018 Data*********************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8612 entries, 0 to 8611
Columns: 15 entries, CAMPUS NAME AND NUMBER to STUDENTS SUSPENDED OUT OF SCHOOL
dtypes: float64(14), object(1)
memory usage: 1009.3+ KB
*********************************After: disciplineStats2018 Data**************************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 8612 entries, 0 to 8611
Columns: 17 entries, CAMPUS NAME AND NUMBER to DISTRICT NAME AND NUMBER
dtypes: float64(14), int64(1), object(2)
memory usage: 1.2+ MB


### This is a Snapshot of the Pivoted Dataset used for Generating Rankings

In [74]:
#Look at the new merged data layout
disciplineStats2018.head()

,CAMPUS NAME AND NUMBER,CAMPUS CUMULATIVE YEAR END ENROLLMENT,CAMPUS DISCIPLINE POPULATION,CAMPUS DISCIPLINE RECORD COUNT,COUNT OF STUDENTS EXPELLED,COUNT OF STUDENTS EXPELLED TO JJAEP,COUNT OF STUDENTS REMOVED TO A DAEP,COUNT OF STUDENTS SUSPENDED IN SCHOOL,DISCRETIONARY DAEP REMOVALS,DISCRETIONARY EXPULSIONS,DISCRETIONARY EXPULSIONS TO JJAEP,MANDATORY DAEP REMOVALS,MANDATORY EXPULSIONS,MANDATORY EXPULSIONS TO JJAEP,STUDENTS SUSPENDED OUT OF SCHOOL,REGION,DISTRICT NAME AND NUMBER
0,21ST CENTURY EARLY LEARNING FOUNDA 108913118,615.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,WESLACO ISD 108913
1,3D ACADEMY 108902006,262.0,42.0,92.0,NaN,NaN,11.0,15.0,NaN,NaN,NaN,14.0,NaN,NaN,32.0,1,DONNA ISD 108902
2,6TH GRADE CENTER 116905108,400.0,83.0,181.0,NaN,NaN,NaN,66.0,NaN,NaN,NaN,NaN,NaN,NaN,40.0,10,GREENVILLE ISD 116905
3,A & M CONS H S 021901001,1804.0,218.0,553.0,NaN,NaN,31.0,194.0,NaN,NaN,NaN,NaN,NaN,NaN,60.0,6,COLLEGE STATION ISD 021901
4,A & M CONSOLIDATED MIDDLE 021901042,961.0,140.0,514.0,NaN,NaN,20.0,134.0,NaN,NaN,NaN,NaN,NaN,NaN,59.0,6,COLLEGE STATION ISD 021901


In [75]:
#Save a copy of the raw disciplne stats for creating the ditrict level dataset later
rawDisciplineStats2018 = disciplineStats2018

#Save the raw discipline stats without rankings to a csv file
outpath = 'D:/Nathan/HebronSchoolData/RawData/CampusDisciplineStats2018.csv'
rawDisciplineStats2018.to_csv(outpath, sep=',', index=False)

## Creating the Campus Discipline Ranks
* Each of the individual campus raw discipline category counts are looked at as a percentage of "Campus Cumulative Year End Enrollment".
* This answers the general question: "What percentage of our student body falls into a particular discipline category".
* These percentages also allow all Texas Public Schools to be equally ranked based on the normalized disciplinary percentages.
* In some instances, percentages may exceed 100% since a single student could have multiple disciplinary actions taken.  For example, it would be normal to see a DAEP campus with a discipline record percentage > 100%.

In [76]:
#Create and rank disipline percentages 
disciplineStats2018['DisciplinePopPct'] = disciplineStats2018['CAMPUS DISCIPLINE POPULATION'] / \
                                          disciplineStats2018['CAMPUS CUMULATIVE YEAR END ENROLLMENT']

disciplineStats2018['DisciplineRecordCtPct'] =  disciplineStats2018['CAMPUS DISCIPLINE RECORD COUNT'] / \
                                                disciplineStats2018['CAMPUS CUMULATIVE YEAR END ENROLLMENT']
    
disciplineStats2018['ExpelledPct'] =  disciplineStats2018['COUNT OF STUDENTS EXPELLED'] / \
                                      disciplineStats2018['CAMPUS CUMULATIVE YEAR END ENROLLMENT']

disciplineStats2018['JJAEP_Pct'] =  disciplineStats2018['COUNT OF STUDENTS EXPELLED TO JJAEP'] / \
                                   disciplineStats2018['CAMPUS CUMULATIVE YEAR END ENROLLMENT']  

disciplineStats2018['DAEP_Pct'] =  disciplineStats2018['COUNT OF STUDENTS REMOVED TO A DAEP'] / \
                                   disciplineStats2018['CAMPUS CUMULATIVE YEAR END ENROLLMENT']  
    
disciplineStats2018['ISS_Pct'] =  disciplineStats2018['COUNT OF STUDENTS SUSPENDED IN SCHOOL'] / \
                                  disciplineStats2018['CAMPUS CUMULATIVE YEAR END ENROLLMENT'] 

disciplineStats2018['Discr_DAEP_Pct'] =  disciplineStats2018['DISCRETIONARY DAEP REMOVALS'] / \
                                         disciplineStats2018['CAMPUS CUMULATIVE YEAR END ENROLLMENT'] 

disciplineStats2018['Discr_Expul_Pct'] =  disciplineStats2018['DISCRETIONARY EXPULSIONS'] / \
                                          disciplineStats2018['CAMPUS CUMULATIVE YEAR END ENROLLMENT']

disciplineStats2018['Discr_Expul_JJAEP_Pct'] =  disciplineStats2018['DISCRETIONARY EXPULSIONS TO JJAEP'] / \
                                                disciplineStats2018['CAMPUS CUMULATIVE YEAR END ENROLLMENT']

disciplineStats2018['Mand_DAEP_Pct'] = disciplineStats2018['MANDATORY DAEP REMOVALS'] / \
                                       disciplineStats2018['CAMPUS CUMULATIVE YEAR END ENROLLMENT'] 

disciplineStats2018['Mand_Expul_Pct'] = disciplineStats2018['MANDATORY EXPULSIONS'] / \
                                        disciplineStats2018['CAMPUS CUMULATIVE YEAR END ENROLLMENT']

disciplineStats2018['Mand_Expul_JJAEP_Pct'] = disciplineStats2018['MANDATORY EXPULSIONS TO JJAEP'] / \
                                              disciplineStats2018['CAMPUS CUMULATIVE YEAR END ENROLLMENT']

disciplineStats2018['Suspension_Pct'] = disciplineStats2018['STUDENTS SUSPENDED OUT OF SCHOOL'] / \
                                        disciplineStats2018['CAMPUS CUMULATIVE YEAR END ENROLLMENT']

In [77]:
#Rank all of our percentage columns
disciplineStats2018['DisciplinePopPct_Rank'] = disciplineStats2018['DisciplinePopPct'].rank(ascending=False, pct=True)
disciplineStats2018['DisciplineRecordCtPct_Rank'] = disciplineStats2018['DisciplineRecordCtPct'].rank(ascending=False, pct=True)
disciplineStats2018['ExpelledPct_Rank'] = disciplineStats2018['ExpelledPct'].rank(ascending=False, pct=True)
disciplineStats2018['JJAEP_Pct_Rank'] = disciplineStats2018['JJAEP_Pct'].rank(ascending=False, pct=True)
disciplineStats2018['DAEP_Pct_Rank'] = disciplineStats2018['DAEP_Pct'].rank(ascending=False, pct=True)
disciplineStats2018['ISS_Pct_Rank'] = disciplineStats2018['ISS_Pct'].rank(ascending=False, pct=True)
disciplineStats2018['Discr_DAEP_Pct_Rank'] = disciplineStats2018['Discr_DAEP_Pct'].rank(ascending=False, pct=True)
disciplineStats2018['Discr_Expul_Pct_Rank'] = disciplineStats2018['Discr_Expul_Pct'].rank(ascending=False, pct=True)
disciplineStats2018['Discr_Expul_JJAEP_Pct_Rank'] = disciplineStats2018['Discr_Expul_JJAEP_Pct'].rank(ascending=False, pct=True)
disciplineStats2018['Mand_DAEP_Pct_Rank'] = disciplineStats2018['Mand_DAEP_Pct'].rank(ascending=False, pct=True)    
disciplineStats2018['Mand_Expul_Pct_Rank'] = disciplineStats2018['Mand_Expul_Pct'].rank(ascending=False, pct=True)  
disciplineStats2018['Mand_Expul_JJAEP_Pct_Rank'] = disciplineStats2018['Mand_Expul_JJAEP_Pct'].rank(ascending=False, pct=True)
disciplineStats2018['Suspension_Pct_Rank'] = disciplineStats2018['Suspension_Pct'].rank(ascending=False, pct=True)

#Create a final blended rank using all of the rank columns 
#disciplineStats2018['Blended_Rank'] = \
#disciplineStats2018[['DisciplinePopPct_Rank','DisciplineRecordCtPct_Rank','ExpelledPct_Rank',
#                     'JJAEP_Pct_Rank', 'DAEP_Pct_Rank', 'ISS_Pct_Rank', 'Discr_DAEP_Pct_Rank',
#                     'Discr_Expul_Pct_Rank','Discr_Expul_JJAEP_Pct_Rank','Mand_DAEP_Pct_Rank',
#                     'Mand_Expul_Pct_Rank', 'Mand_Expul_JJAEP_Pct_Rank', 'Suspension_Pct_Rank'
#                    ]].mean(axis=1)

In [78]:
#Save the final rankings data to a csv file
outpath = 'D:/Nathan/HebronSchoolData/RawData/CampusDisciplineStatsAndRanks2018.csv'
disciplineStats2018.to_csv(outpath, sep=',', index=False)

# District Level Discipline Ranks Dataset
* This dataset ranks all school discipline trends in the State of Texas at the district level

In [79]:
#Fields to summarize at the district level
colsToSum = ['CAMPUS CUMULATIVE YEAR END ENROLLMENT',
               'CAMPUS DISCIPLINE POPULATION','CAMPUS DISCIPLINE RECORD COUNT','COUNT OF STUDENTS EXPELLED',
               'COUNT OF STUDENTS EXPELLED TO JJAEP', 'COUNT OF STUDENTS REMOVED TO A DAEP',
               'COUNT OF STUDENTS SUSPENDED IN SCHOOL', 'DISCRETIONARY DAEP REMOVALS', 'DISCRETIONARY EXPULSIONS',
               'DISCRETIONARY EXPULSIONS TO JJAEP', 'MANDATORY DAEP REMOVALS', 'MANDATORY EXPULSIONS',
               'MANDATORY EXPULSIONS TO JJAEP','STUDENTS SUSPENDED OUT OF SCHOOL' 
              ]

#Sum all disciplne columns by school district
distDisciplineStats2018 = rawDisciplineStats2018.groupby(['REGION', 'DISTRICT NAME AND NUMBER'])[colsToSum].sum()

#Make our index a column for csv file
distDisciplineStats2018.reset_index(inplace=True)

#Update column names to district level
distDisciplineStats2018.columns = [col.replace('CAMPUS ','') for col in distDisciplineStats2018.columns]
distDisciplineStats2018.columns = [col.replace('COUNT OF ','') for col in distDisciplineStats2018.columns]

In [80]:
#Save the raw district level discipline stats without rankings to a csv file
outpath = 'D:/Nathan/HebronSchoolData/RawData/DistrictDisciplineStats2018.csv'
distDisciplineStats2018.to_csv(outpath, sep=',', index=False)

## Creating the District Disipline Ranks

In [82]:
#Create and rank disipline percentages 
distDisciplineStats2018['DisciplinePopPct'] = distDisciplineStats2018['DISCIPLINE POPULATION'] / \
                                          distDisciplineStats2018['CUMULATIVE YEAR END ENROLLMENT']

distDisciplineStats2018['DisciplineRecordCtPct'] =  distDisciplineStats2018['DISCIPLINE RECORD COUNT'] / \
                                                distDisciplineStats2018['CUMULATIVE YEAR END ENROLLMENT']
    
distDisciplineStats2018['ExpelledPct'] =  distDisciplineStats2018['STUDENTS EXPELLED'] / \
                                      distDisciplineStats2018['CUMULATIVE YEAR END ENROLLMENT']

distDisciplineStats2018['JJAEP_Pct'] =  distDisciplineStats2018['STUDENTS EXPELLED TO JJAEP'] / \
                                   distDisciplineStats2018['CUMULATIVE YEAR END ENROLLMENT']  

distDisciplineStats2018['DAEP_Pct'] =  distDisciplineStats2018['STUDENTS REMOVED TO A DAEP'] / \
                                   distDisciplineStats2018['CUMULATIVE YEAR END ENROLLMENT']  
    
distDisciplineStats2018['ISS_Pct'] =  distDisciplineStats2018['STUDENTS SUSPENDED IN SCHOOL'] / \
                                  distDisciplineStats2018['CUMULATIVE YEAR END ENROLLMENT'] 

distDisciplineStats2018['Discr_DAEP_Pct'] =  distDisciplineStats2018['DISCRETIONARY DAEP REMOVALS'] / \
                                         distDisciplineStats2018['CUMULATIVE YEAR END ENROLLMENT'] 

distDisciplineStats2018['Discr_Expul_Pct'] =  distDisciplineStats2018['DISCRETIONARY EXPULSIONS'] / \
                                          distDisciplineStats2018['CUMULATIVE YEAR END ENROLLMENT']

distDisciplineStats2018['Discr_Expul_JJAEP_Pct'] =  distDisciplineStats2018['DISCRETIONARY EXPULSIONS TO JJAEP'] / \
                                                distDisciplineStats2018['CUMULATIVE YEAR END ENROLLMENT']

distDisciplineStats2018['Mand_DAEP_Pct'] = distDisciplineStats2018['MANDATORY DAEP REMOVALS'] / \
                                       distDisciplineStats2018['CUMULATIVE YEAR END ENROLLMENT'] 

distDisciplineStats2018['Mand_Expul_Pct'] = distDisciplineStats2018['MANDATORY EXPULSIONS'] / \
                                        distDisciplineStats2018['CUMULATIVE YEAR END ENROLLMENT']

distDisciplineStats2018['Mand_Expul_JJAEP_Pct'] = distDisciplineStats2018['MANDATORY EXPULSIONS TO JJAEP'] / \
                                              distDisciplineStats2018['CUMULATIVE YEAR END ENROLLMENT']

distDisciplineStats2018['Suspension_Pct'] = distDisciplineStats2018['STUDENTS SUSPENDED OUT OF SCHOOL'] / \
                                        distDisciplineStats2018['CUMULATIVE YEAR END ENROLLMENT']

In [83]:
#Rank all of our percentage columns
distDisciplineStats2018['DisciplinePopPct_Rank'] = distDisciplineStats2018['DisciplinePopPct'].rank(ascending=False, pct=True)
distDisciplineStats2018['DisciplineRecordCtPct_Rank'] = distDisciplineStats2018['DisciplineRecordCtPct'].rank(ascending=False, pct=True)
distDisciplineStats2018['ExpelledPct_Rank'] = distDisciplineStats2018['ExpelledPct'].rank(ascending=False, pct=True)
distDisciplineStats2018['JJAEP_Pct_Rank'] = distDisciplineStats2018['JJAEP_Pct'].rank(ascending=False, pct=True)
distDisciplineStats2018['DAEP_Pct_Rank'] = distDisciplineStats2018['DAEP_Pct'].rank(ascending=False, pct=True)
distDisciplineStats2018['ISS_Pct_Rank'] = distDisciplineStats2018['ISS_Pct'].rank(ascending=False, pct=True)
distDisciplineStats2018['Discr_DAEP_Pct_Rank'] = distDisciplineStats2018['Discr_DAEP_Pct'].rank(ascending=False, pct=True)
distDisciplineStats2018['Discr_Expul_Pct_Rank'] = distDisciplineStats2018['Discr_Expul_Pct'].rank(ascending=False, pct=True)
distDisciplineStats2018['Discr_Expul_JJAEP_Pct_Rank'] = distDisciplineStats2018['Discr_Expul_JJAEP_Pct'].rank(ascending=False, pct=True)
distDisciplineStats2018['Mand_DAEP_Pct_Rank'] = distDisciplineStats2018['Mand_DAEP_Pct'].rank(ascending=False, pct=True)    
distDisciplineStats2018['Mand_Expul_Pct_Rank'] = distDisciplineStats2018['Mand_Expul_Pct'].rank(ascending=False, pct=True)  
distDisciplineStats2018['Mand_Expul_JJAEP_Pct_Rank'] = distDisciplineStats2018['Mand_Expul_JJAEP_Pct'].rank(ascending=False, pct=True)
distDisciplineStats2018['Suspension_Pct_Rank'] = distDisciplineStats2018['Suspension_Pct'].rank(ascending=False, pct=True)

In [84]:
#Save the district level discipline stats and rankings to a csv file
outpath = 'D:/Nathan/HebronSchoolData/RawData/DistrictDisciplineStatsAndRanks2018.csv'
distDisciplineStats2018.to_csv(outpath, sep=',', index=False)